In [6]:
# Importation des librairies
from my_function import ( 
    sel_classif_pixel,
    report_from_dict_to_df,
    supprimer_dossier_non_vide,
    rasterization)
import os
import matplotlib.pyplot as plt
from osgeo import gdal
import geopandas as gpd
import numpy as np
import pandas as pd
import sys
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import (train_test_split, KFold,
    StratifiedKFold, GroupKFold, StratifiedGroupKFold)
from sklearn.metrics import (confusion_matrix, classification_report,
    accuracy_score)
sys.path.append('/home/onyxia/work/libsigma')
import read_and_write as rw
import classification as cla
import plots

In [7]:
# Création du dossier de sauvegarde temporaire et paramètres
racine = "/home/onyxia/work"
output_dir = os.path.join(racine,"output_classif")
os.makedirs(output_dir, exist_ok=True) # Création d'un dossier output temporaire

path_sample = os.path.join(racine,"Depot_Git/results/data/sample/Sample_BD_foret_T31TCJ.shp")
path_sample_px = os.path.join(output_dir,"sample_classif_px.shp")

sample_rasterized = os.path.join(output_dir,"rasterized_sample.tif")
path_image_3b = os.path.join(racine,"Depot_Git/results/data/img_pretraitees/Serie_temp_S2_3_bands.tif")
path_image_10b = os.path.join(racine,"Depot_Git/results/data/img_pretraitees/Serie_temp_S2_10_bands.tif")
path_image_allbands = os.path.join(racine,"Depot_Git/results/data/img_pretraitees/Serie_temp_S2_allbands.tif")

path_sample_px_centroid = os.path.join(output_dir,"sample_px_centroid.shp")
path_sample_px_id = os.path.join(output_dir,"sample_px_id.shp")
path_rasterized_sample_id = os.path.join(output_dir, "rasterized_sample_id.tif")

In [8]:
# Save d'un vecteur echantillons avec que les données pour classif pixel
sample = gpd.read_file(path_sample)
sample_px = sel_classif_pixel(sample[['Code',"geometry"]])
sample_px.to_file(path_sample_px)

In [9]:
rasterization(
    in_vector=path_sample_px,
    out_image=sample_rasterized,
    field_name='Code',
    ref_image=path_image_allbands
)

0...10...20...30...40...50...60...70...80...90...100 - done.


In [ ]:
# Méthode stratifiée simple
# 1 --- define parameters
# inputs
sample_filename = sample_rasterized
image_filename = path_image_3b

# Sample parameters
#test_size = 0.7
nb_iter = 30
nb_folds = 5
is_point = False
# if is_point is True
field_name = 'Code'

# outputs
suffix = '_CV{}fold_stratifed_x{}times'.format(nb_folds, nb_iter)
out_folder = output_dir
out_classif = os.path.join(out_folder, 'ma_classif{}.tif'.format(suffix))
out_matrix = os.path.join(out_folder, 'ma_matrice{}.png'.format(suffix))
out_qualite = os.path.join(out_folder, 'mes_qualites{}.png'.format(suffix))

print("Extraction des échantillons")
# 2 --- extract samples
if not is_point :
    X, Y, t = cla.get_samples_from_roi(image_filename, sample_filename)
else :
    # get X
    list_row, list_col = rw.get_row_col_from_file(sample_filename, image_filename)
    image = rw.load_img_as_array(image_filename)
    X = image[(list_row, list_col)]

    # get Y
    gdf = gpd.read_file(sample_filename)
    Y = gdf.loc[:, field_name].values
    Y = np.atleast_2d(Y).T

list_cm = []
list_accuracy = []
list_report = []

print("KFold début")
kf = StratifiedKFold(n_splits=nb_folds)
for train, test in kf.split(X, Y):
    print(train,test)
    X_train, X_test = X[train], X[test]
    Y_train, Y_test = Y[train], Y[test]

    print("Entrainement")
    # 3 --- Train
    rfc = RandomForestClassifier(
        max_depth = 50,
        oob_score = True,
        max_samples = 0.75,
        class_weight = 'balanced',
        n_jobs = -1
    )
    rfc.fit(X_train, Y_train[:,0])

    print("Predict")
    # 4 --- Test
    Y_predict = rfc.predict(X_test)

    # compute quality
    list_cm.append(confusion_matrix(Y_test, Y_predict))
    list_accuracy.append(accuracy_score(Y_test, Y_predict))
    report = classification_report(Y_test, Y_predict,

                                   labels=np.unique(Y_predict), output_dict=True)

    # store them
    list_report.append(report_from_dict_to_df(report))

# compute mean of cm
array_cm = np.array(list_cm)
mean_cm = array_cm.mean(axis=0)

# compute mean and std of overall accuracy
array_accuracy = np.array(list_accuracy)
mean_accuracy = array_accuracy.mean()
std_accuracy = array_accuracy.std()

# compute mean and std of classification report
array_report = np.array(list_report)
mean_report = array_report.mean(axis=0)
std_report = array_report.std(axis=0)
a_report = list_report[0]
mean_df_report = pd.DataFrame(mean_report, index=a_report.index,
                              columns=a_report.columns)
std_df_report = pd.DataFrame(std_report, index=a_report.index,
                             columns=a_report.columns)

# Display confusion matrix
plots.plot_cm(mean_cm, np.unique(Y_predict))
plt.savefig(out_matrix, bbox_inches='tight')

# Display class metrics
fig, ax = plt.subplots(figsize=(10, 7))
ax = mean_df_report.T.plot.bar(ax=ax, yerr=std_df_report.T, zorder=2)
ax.set_ylim(0.5, 1)
_ = ax.text(1.5, 0.95, 'OA : {:.2f} +- {:.2f}'.format(mean_accuracy,
                                                      std_accuracy),
            fontsize=14)
ax.set_title('Class quality estimation')

# custom : cuteness
# background color
ax.set_facecolor('ivory')
# labels
x_label = ax.get_xlabel()
ax.set_xlabel(x_label, fontdict={'fontname': 'Sawasdee'}, fontsize=14)
y_label = ax.get_ylabel()
ax.set_ylabel(y_label, fontdict={'fontname': 'Sawasdee'}, fontsize=14)
# borders
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["left"].set_visible(False)
ax.tick_params(axis='x', colors='darkslategrey', labelsize=14)
ax.tick_params(axis='y', colors='darkslategrey', labelsize=14)
# grid
ax.minorticks_on()
ax.yaxis.grid(which='major', color='darkgoldenrod', linestyle='--',
              linewidth=0.5, zorder=1)
ax.yaxis.grid(which='minor', color='darkgoldenrod', linestyle='-.',
              linewidth=0.3, zorder=1)
plt.savefig(out_qualite, bbox_inches='tight')

In [10]:
# Méthode stratifiée groupée 

L = [i+1 for i in range(sample_px.shape[0])]  # Création des id de polygones
sample_px_id = sample_px.copy()
sample_px_id['id'] = L
sample_px_id = sample_px_id[['id','geometry']]
sample_px_id.to_file(path_sample_px_id)

rasterization(
    in_vector=path_sample_px_id,
    out_image=path_rasterized_sample_id,
    field_name="id",
    ref_image=path_image_allbands
)

0...10...20...30...40...50...60...70...80...90...100 - done.


In [11]:
# 1 --- define parameters
# inputs
sample_filename = sample_rasterized
image_filename = path_image_allbands
id_filename = os.path.join(output_dir, "rasterized_sample_id.tif")

# Sample parameters
#test_size = 0.7
nb_iter = 30
nb_folds = 5
is_point = False
# if is_point is True
field_name = 'Code'

# outputs
suffix = '_CV{}folds_stratified_group_x{}times'.format(nb_folds, nb_iter)
out_folder = "/home/onyxia/work"
out_classif = os.path.join(out_folder, 'ma_classif{}.tif'.format(suffix))
out_matrix = os.path.join(out_folder, 'ma_matrice{}.png'.format(suffix))
out_qualite = os.path.join(out_folder, 'mes_qualites{}.png'.format(suffix))


# 2 --- extract samples
if not is_point :
    X, Y, t = cla.get_samples_from_roi(image_filename, sample_filename)
    _, groups, _ = cla.get_samples_from_roi(image_filename, id_filename)
else :
    # get X
    list_row, list_col = rw.get_row_col_from_file(sample_filename, image_filename)
    image = rw.load_img_as_array(image_filename)
    X = image[(list_row, list_col)]

    # get Y
    gdf = gpd.read_file(sample_filename)
    Y = gdf.loc[:, field_name].values
    Y = np.atleast_2d(Y).T

list_cm = []
list_accuracy = []
list_report = []
groups = np.squeeze(groups)

# Iter on stratified K fold
for i in range(nb_iter):
    print (f"Début de la {i+1} itération")
    kf = StratifiedGroupKFold(n_splits=nb_folds, shuffle=True)
    for train, test in kf.split(X, Y, groups=groups):
        X_train, X_test = X[train], X[test]
        Y_train, Y_test = Y[train], Y[test]

        # 3 --- Train
        rfc = RandomForestClassifier(
            max_depth = 50,
            oob_score = True,
            max_samples = 0.75,
            class_weight = 'balanced',
            n_jobs = -1
        )
        rfc.fit(X_train, Y_train[:,0])

        # 4 --- Test
        Y_predict = rfc.predict(X_test)

        # compute quality
        list_cm.append(confusion_matrix(Y_test, Y_predict))
        list_accuracy.append(accuracy_score(Y_test, Y_predict))
        report = classification_report(Y_test, Y_predict,
                                        labels=np.unique(Y_predict),
                                        output_dict=True,
                                        zero_division = 0)

        # store them
        list_report.append(report_from_dict_to_df(report))

Début de la 1 itération


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


{'11': {'precision': 0.7508408796895213, 'recall': 0.04635042325507108, 'f1-score': 0.0873110191801429, 'support': 62610.0}, '12': {'precision': 0.9316018226588596, 'recall': 0.9978058726211104, 'f1-score': 0.9635680129892457, 'support': 2041814.0}, '13': {'precision': 0.36538461538461536, 'recall': 0.0008868972599542547, 'f1-score': 0.001769499417927823, 'support': 21423.0}, '14': {'precision': 0.87, 'recall': 0.004967076466334259, 'f1-score': 0.00987775801385157, 'support': 52546.0}, '22': {'precision': 0.22635135135135134, 'recall': 0.05929203539823009, 'f1-score': 0.09396914446002805, 'support': 2260.0}, '23': {'precision': 0.15440350606734646, 'recall': 0.7068965517241379, 'f1-score': 0.25344780840488024, 'support': 9918.0}, '24': {'precision': 0.8100477267018337, 'recall': 0.3331955695156224, 'f1-score': 0.47217300906335563, 'support': 48392.0}, 'micro avg': {'precision': 0.9143784148768562, 'recall': 0.9217593144683499, 'f1-score': 0.9180540298162878, 'support': 2238963.0}, 'mac

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


{'11': {'precision': 0.7436315467699205, 'recall': 0.059009023577735374, 'f1-score': 0.10934152370964118, 'support': 61838.0}, '12': {'precision': 0.9347146297993452, 'recall': 0.9970446727967437, 'f1-score': 0.9648740823705811, 'support': 2035646.0}, '13': {'precision': 0.3793103448275862, 'recall': 0.001026933669420716, 'f1-score': 0.002048321772729389, 'support': 21423.0}, '14': {'precision': 0.4864864864864865, 'recall': 0.0010395010395010396, 'f1-score': 0.002074569238748343, 'support': 51948.0}, '22': {'precision': 0.2926829268292683, 'recall': 0.10783055198973042, 'f1-score': 0.1575984990619137, 'support': 1558.0}, '23': {'precision': 0.2453693801599863, 'recall': 0.4996080480794356, 'f1-score': 0.3291066613116071, 'support': 11481.0}, '24': {'precision': 0.5676107571647848, 'recall': 0.5867330304940338, 'f1-score': 0.5770135088919288, 'support': 46009.0}, 'micro avg': {'precision': 0.919161948348203, 'recall': 0.9266116059756859, 'f1-score': 0.922871743519644, 'support': 222990

...

{'11': {'precision': 0.7154036243822076, 'recall': 0.05640342901675542, 'f1-score': 0.10456296653021911, 'support': 61592.0}, '12': {'precision': 0.931667994819974, 'recall': 0.9979239893800501, 'f1-score': 0.9636584830174988, 'support': 2046714.0}, '13': {'precision': 0.37755102040816324, 'recall': 0.0017271157167530224, 'f1-score': 0.0034385019283490544, 'support': 21423.0}, '14': {'precision': 0.8154362416107382, 'recall': 0.004629365034005829, 'f1-score': 0.009206463467767906, 'support': 52491.0}, '22': {'precision': 0.14277389277389277, 'recall': 0.1084070796460177, 'f1-score': 0.12323943661971831, 'support': 2260.0}, '23': {'precision': 0.18847457627118644, 'recall': 0.013866028230834456, 'f1-score': 0.025831629808585766, 'support': 20049.0}, '24': {'precision': 0.4200541606569923, 'recall': 0.6197290137664481, 'f1-score': 0.500719298245614, 'support': 46054.0}, 'micro avg': {'precision': 0.9147631901883758, 'recall': 0.9221090712939713, 'f1-score': 0.9184214421941393, 'support':

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


{'11': {'precision': 0.7357664233576642, 'recall': 0.056791474839831296, 'f1-score': 0.10544405756384775, 'support': 62122.0}, '12': {'precision': 0.9328369055953223, 'recall': 0.9974464414203726, 'f1-score': 0.9640603835184834, 'support': 2033241.0}, '13': {'precision': 0.41379310344827586, 'recall': 0.0011202912757316903, 'f1-score': 0.002234532842977515, 'support': 21423.0}, '14': {'precision': 0.2978723404255319, 'recall': 0.000538006302359542, 'f1-score': 0.0010740726533430512, 'support': 52044.0}, '22': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1088.0}, '23': {'precision': 0.17549470915286663, 'recall': 0.5654435134354233, 'f1-score': 0.2678559207974998, 'support': 10383.0}, '24': {'precision': 0.6042605105105106, 'recall': 0.3995325653064179, 'f1-score': 0.481018962361642, 'support': 48349.0}, 'micro avg': {'precision': 0.9154742262397279, 'recall': 0.9228981670518026, 'f1-score': 0.9191712065117253, 'support': 2228650.0}, 'macro avg': {'precision': 0.4514319

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


{'11': {'precision': 0.6311188811188811, 'recall': 0.06319525120549994, 'f1-score': 0.11488666116969723, 'support': 62837.0}, '12': {'precision': 0.92513540982024, 'recall': 0.998928899689191, 'f1-score': 0.9606170567457442, 'support': 2053029.0}, '14': {'precision': 0.6303724928366762, 'recall': 0.00428741254652817, 'f1-score': 0.008516898300491658, 'support': 51313.0}, '22': {'precision': 0.034031826059555695, 'recall': 0.09557522123893805, 'f1-score': 0.050191704426629485, 'support': 2260.0}, '23': {'precision': 0.5015920070267896, 'recall': 0.469792791403157, 'f1-score': 0.485171910261516, 'support': 19449.0}, '24': {'precision': 0.7200079317866349, 'recall': 0.44196048200592364, 'f1-score': 0.5477172164119066, 'support': 49294.0}, 'micro avg': {'precision': 0.9156848828333665, 'recall': 0.9320779096606084, 'f1-score': 0.9238086782201795, 'support': 2238182.0}, 'macro avg': {'precision': 0.5737097581081295, 'recall': 0.34562334301487296, 'f1-score': 0.3611835745526642, 'support': 2

In [14]:
import json

# Sauvegarder la liste dans un fichier JSON
# with open(os.path.join(racine,"list_cm.json"), "w", encoding="utf-8") as fichier:
#     json.dump(list_cm, fichier, indent=4, ensure_ascii=False)
# with open(os.path.join(racine,"list_accuracy.json"), "w", encoding="utf-8") as fichier:
#     json.dump(list_accuracy, fichier, indent=4, ensure_ascii=False)
list_json = [df.to_dict(orient="records") for df in list_report]
with open(os.path.join(racine,"list_report.json"), "w", encoding="utf-8") as fichier:
    json.dump(list_json, fichier, indent=4, ensure_ascii=False)

In [15]:
import json
# Open list_report
with open(os.path.join(racine,"list_report.json"), "r", encoding="utf-8") as fichier:
    list_json = json.load(fichier)
    fichier = None
list_report = [pd.DataFrame(i) for i in list_json]

In [16]:
# Stratégie d'évitement : supprimer les tableaux qui n'ont pas toutes les classes
list_report_2 = []
list_cm_2 = []
try :
    list_report
except NameError:
    print('list_report is not defined')
else :
    for report in list_report:
        nb_report = len(report.keys())
        if nb_report == 9 :
            list_report_2.append(report)
    print(len(list_report_2))
try :
    list_cm
except NameError:
    print('list_cm is not defined')
else :
    for cm in list_cm:
        nb_cm = len(cm)
        if nb_cm == 9:
            list_cm_2.append(cm)
    print(len(list_cm_2))
finally:
  print("The 'try except' is finished")

0
30
The 'try except' is finished


In [ ]:
# compute mean of cm
array_cm = np.array(list_cm_2)
mean_cm = array_cm.mean(axis=0)

# compute mean and std of overall accuracy
array_accuracy = np.array(list_accuracy)
mean_accuracy = array_accuracy.mean()
std_accuracy = array_accuracy.std()

# compute mean and std of classification report
array_report = np.array(list_report_2)
mean_report = array_report.mean(axis=0)
std_report = array_report.std(axis=0)
a_report = list_report[0]
mean_df_report = pd.DataFrame(mean_report, index=a_report.index,
                              columns=a_report.columns)
std_df_report = pd.DataFrame(std_report, index=a_report.index,
                             columns=a_report.columns)

# Display confusion matrix
plots.plot_cm(mean_cm, np.unique(Y_predict))
plt.savefig(out_matrix, bbox_inches='tight')

# Display class metrics
fig, ax = plt.subplots(figsize=(10, 7))
ax = mean_df_report.T.plot.bar(ax=ax, yerr=std_df_report.T, zorder=2)
ax.set_ylim(0.5, 1)
_ = ax.text(1.5, 0.95, 'OA : {:.2f} +- {:.2f}'.format(mean_accuracy,
                                                      std_accuracy),
            fontsize=14)
ax.set_title('Class quality estimation')

# custom : cuteness
# background color
ax.set_facecolor('ivory')
# labels
x_label = ax.get_xlabel()
ax.set_xlabel(x_label, fontdict={'fontname': 'Sawasdee'}, fontsize=14)
y_label = ax.get_ylabel()
ax.set_ylabel(y_label, fontdict={'fontname': 'Sawasdee'}, fontsize=14)
# borders
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["left"].set_visible(False)
ax.tick_params(axis='x', colors='darkslategrey', labelsize=14)
ax.tick_params(axis='y', colors='darkslategrey', labelsize=14)
# grid
ax.minorticks_on()
ax.yaxis.grid(which='major', color='darkgoldenrod', linestyle='--',
              linewidth=0.5, zorder=1)
ax.yaxis.grid(which='minor', color='darkgoldenrod', linestyle='-.',
              linewidth=0.3, zorder=1)
plt.savefig(out_qualite, bbox_inches='tight')

In [10]:
# Ecriture de la classif

X_img, _, t_img = cla.get_samples_from_roi(path_image_allbands, path_image_allbands)
Y_predict = rfc.predict(X_img)
image_filename = path_image_allbands

# Get image dimension
ds = rw.open_image(image_filename)
nb_row, nb_col, _ = rw.get_image_dimension(ds)

#initialization of the array
img = np.zeros((nb_row, nb_col, 1), dtype='uint8')
#np.Y_predict

img[t_img[0], t_img[1], 0] = Y_predict
image_filename =path_image_allbands

out_filename = '/home/onyxia/work/Depot_Git/results/data/classif/carte_essences_echelle_pixel.tif'

# get reference dataset
ds = rw.open_image(image_filename)

# just replace the number of band (1 instead of 3)
rw.write_image(out_filename, img, data_set=ds, gdal_dtype=gdal.GDT_Byte,
                transform=None, projection=None, driver_name=None,
                nb_col=None, nb_ligne=None, nb_band=1)

In [38]:
## Nettoyage des dossiers
supprimer_dossier_non_vide(output_dir)